<a href="https://colab.research.google.com/github/suwatoh/Python-learning/blob/main/116_%E3%83%95%E3%82%A1%E3%82%A4%E3%83%AB%E3%81%A8%E3%83%87%E3%82%A3%E3%83%AC%E3%82%AF%E3%83%88%E3%83%AA%E3%81%B8%E3%81%AE%E3%82%A2%E3%82%AF%E3%82%BB%E3%82%B9.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

ファイルとディレクトリへのアクセス
==================================

パスの表現
----------

### 絶対パス・相対パス

**絶対パス**（absolute path）とは、ルートディレクトリからの完全な表現で指定したパスのことである。フルパス（full path）とも呼ばれる。

**相対パス**（relative path）とは、ユーザが現在作業しているフォルダ（カレントディレクトリ）を表す記号 `.` と、直接の上位パスを表す記号 `..` を用いてなんらかの任意のフォルダから目的のフォルダまでの経路を表すパスのことである。

### globパターン

**glob パターン**とは、Unix 系環境においてワイルドカードでファイル名のセットを指定するパターンのことである。ワイルドカードとして使用できる記号は以下の通り。

| ワイルドカード | 意味 |
|:---|:---|
| `*` | 0 文字以上の任意の文字列にマッチする |
| `?` | 任意の 1 文字にマッチする |
| `[seq]` | `seq` にある任意の文字にマッチする。正規表現と同じ書式 |
| `[!seq]` | `seq` にない任意の文字にマッチする。正規表現の `[^seq]` と同じ |

たとえば、`[0-9]` は任意のアラビア数字にマッチする。`[!0-9]` はアラビア数字以外の任意の文字にマッチする。

標準ライブラリの `glob` モジュールでは、ディレクトリのワイルドカード検索からファイルのリストを生成するための関数を提供している:

In [ ]:
import glob
glob.glob('./*/*.csv')

['./sample_data/california_housing_train.csv',
 './sample_data/mnist_test.csv',
 './sample_data/mnist_train_small.csv',
 './sample_data/california_housing_test.csv']

path-like オブジェクト
----------------------

Python で **path-like オブジェクト** とは、パスを表す文字列（str）、バイト列（bytes）、または `os.PathLike` インターフェースを実装したオブジェクトのことをいう。

`os.PathLike` は抽象基底クラスで、唯一の抽象メソッド `__fspath__()` を持つ。`__fspath__()` は、このオブジェクトが表現するファイルシステムパスを返す。戻り値は `str` か `bytes` だけが許される。

`os.fspath(path)` 関数は、もし文字列やバイト列が渡された場合は、変更せずにそのまま返す。そうでなければ、`path.__fspath__()` が呼び出され、その戻り値が文字列かバイト列であれば、その値を返す。他のすべてのケースでは `TypeError` 例外が発生する。

In [ ]:
import os

class MyFile(os.PathLike):
    def __fspath__(self):
        return "myfile"

f = MyFile()
assert os.fspath(f) == "myfile"

パスオブジェクト
----------------

### 純粋パスと具象パス

標準ライブラリの `pathlib` モジュールは、`os.PathLike` インターフェースを実装するクラスを提供する。それらは、次の 2 種類に分類される。

  * **純粋パス**（pure path）:  
  ファイルシステムにアクセスしない、純粋に「文字列としてのパス」を扱うクラス。
      * `PurePath`
      * `PureWindowsPath`
      * `PurePosixPath`
  * **具象パス**（concrete path）:  
  実際のファイルシステムにアクセスできるクラス。
      * `Path`
      * `WindowsPath`
      * `PosixPath`

継承関係は次の図のとおり（公式ドキュメントより引用）。

![](https://docs.python.org/ja/3/_images/pathlib-inheritance.png)

基本的には具象パス `Path` を使用する。具象パスのオブジェクトが作成されると、それは `WindowsPath` のインスタンスか `PosixPath` のインスタンスのどちらかであり、コードが実行されているプラットフォームによって自動的に決まる。Unix 上で実行しているときに `WindowsPath` のインスタンスを作成することはできない（逆もまた同じ）。

一方、`PureWindowsPath` および `PurePosixPath` は、あらゆるシステムコールを行わないため、起動しているシステムにかかわらずインスタンスを作成できる。

### パスオブジェクトの性質

■ path-like オブジェクト

`PurePath`（を継承するクラス）は `os.PathLike` インターフェースを実装しているので、そのインスタンスは path-like オブジェクトであり、path-like オブジェクトに対応する関数（たとえば組み込み関数 `open()`）の引数に渡すことができる。`__fspath__()` の実装は、単に `str(self)` を呼び出して戻り値を返すだけである。したがって、`str(path)` と `os.fspath(path)` は完全に同じ値となる。

■ イミュータブル

パスオブジェクトはイミュータブル（変更不可能）であり、ハッシュ可能である。

■ 等価性

パスオブジェクト同士の比較 `==`、`!=` が可能である。2 つのパスオブジェクトが表現するパスが完全に一致するとき、`==` の評価は `*WindowsPath` 同士または `*PosixPath` 同士で `True` となる。大文字、小文字を区別しない。絶対パスと相対パスは区別される。`*WindowsPath` と `*PosixPath` の組み合わせでは、`==` は常に `False` となる。

### パスオブジェクトの作成

``` python
pathlib.Path(*pathsegments)
```

コンストラクタの引数 `pathsegments` の各要素は、パスの構成要素を表す文字列か、path-like オブジェクトである必要がある。それらは実際のファイルシステムに存在しないものでもよい。引数を省略すると、カレントディレクトリパス `Path(".")` とみなす。

In [ ]:
from pathlib import Path
import os

# 相対パスを指定
path = Path('setup.py')
assert str(path) == os.fspath(path) == "setup.py"

# 絶対パスを指定
path = Path('/foo/some/path/bar')
assert str(path) == os.fspath(path) == "/foo/some/path/bar"

# パスを構成する要素を指定
mydir = "bar"
assert Path("/", "foo", "some/path", mydir) == path

# 引数を省略--カレントディレクトリパスとみなされる
assert Path() == Path(".")

除算演算子 `/` を使って、子パスを作成することができる:

In [ ]:
from pathlib import Path

path = Path("/etc")
assert (path / "init.d" / "apache2")  == (path / Path("init.d", "apache2"))  == Path("/etc/init.d/apache2")

また、以下のクラスメソッドを呼び出すことでも、パスオブジェクトを作成できる。

| クラスメソッド | 機能 |
|:---|:---|
| `Path.cwd()` | カレントディレクトリを表すパスオブジェクトを返す |
| `Path.home()` | ユーザーのホームディレクトリを表すパスオブジェクトを返す。ホームディレクトリが解決できない場合は、`RuntimeError` 例外を送出する |

これらのメソッドは、絶対パスを表すパスオブジェクトを作成する。絶対パスと相対パスは、パスの表現としては異なるので、`Path.cwd()` と `Path(".")` は等しくないとされる。

In [ ]:
from pathlib import Path

assert (cwd := Path.cwd()) != Path('.')
print(str(cwd))
print(str(Path('.')))
print(str(Path.home()))

/content
.
/root


### パスオブジェクトのプロパティ

パスオブジェクトのプロパティは、純粋パス `PurePath` から継承しており、すべて読み出し専用である。

| プロパティ | 意味 | 戻り値 |
|:---|:---|:---|
| `parts` | パスを構成する要素からなるタプルを返す | `tuple` |
| `drive` | ドライブを表す文字列を返す。ドライブがなければ空文字列を返す | `str` |
| `root` | ルートを表す文字列を返す。ルートがなければ空文字列を返す | `str` |
| `anchor` | ドライブとルートを結合した文字列を返す | `str` |
| `name` | パス要素の末尾を表す文字列 |`str` |
| `suffix` | 末尾の要素に拡張子（ドット `.` 付き文字列）があればそれを返す。なければ空文字列を返す | `str` |
| `suffixes` | 拡張子のリストを返す（例: `Path('my/library.tar.gz').suffixes` => `['.tar', '.gz']`） | `list` |
| `stem` | 末尾の要素から拡張子を除いたものを返す（例: `Path('my/library.tar.gz').stem` => `'library.tar'`） | `str` |
| `parent` | パスの直接の上位パスを返す | パスオブジェクト |
| `parents` | 上位パスにアクセスできるイミュータブルなシーケンスを返す | パスオブジェクトのシーケンス |

In [ ]:
from pathlib import Path

path = Path("/spam/ham/eggs.py")
print(f"{path=}")
print(f"{path.parts=}")
print(f"{path.drive=}")
print(f"{path.root=}")
print(f"{path.anchor=}")
print(f"{path.name=}")
print(f"{path.suffix=}")
print(f"{path.suffixes=}")
print(f"{path.stem=}")
print(f"{path.parent=}")
print(f"{path.parents[0]=}")
print(f"{path.parents[1]=}")
print(f"{path.parents[2]=}")

path=PosixPath('/spam/ham/eggs.py')
path.parts=('/', 'spam', 'ham', 'eggs.py')
path.drive=''
path.root='/'
path.anchor='/'
path.name='eggs.py'
path.suffix='.py'
path.suffixes=['.py']
path.stem='eggs'
path.parent=PosixPath('/spam/ham')
path.parents[0]=PosixPath('/spam/ham')
path.parents[1]=PosixPath('/spam')
path.parents[2]=PosixPath('/')


プロパティの値は、プラットフォームに依存する。Unix マシン上で Windows のパスを扱いたいときは `PureWindowsPath` のインスタンスを、Windows マシン上で Unix のパスを扱いたいときは `PurePosixPath` のインスタンスを作成して、それによりエミュレーションが可能である。

In [ ]:
from pathlib import PureWindowsPath

path = PureWindowsPath(r"c:\Program Files\UNP")
print(f"{path.drive=}")
print(f"{path.root=}")
print(f"{path.anchor=}")

path.drive='c:'
path.root='\\'
path.anchor='c:\\'


### パスのチェック・変換を行うメソッド

| メソッド | 機能 | 戻り値 |
|:---|:---|:---|
| `is_absolute()` | パスが絶対パスである場合に `True` を返す | `bool` |
| `is_relative_to(other)` | パスが `other` パスに対して相対である場合に `True` を返す | `bool` |
| `match(pattern)` | glob 形式の `pattern` と一致する場合に `True` を返す | `bool` |
| `joinpath(*pathsegments)` | 与えられた引数を順番に結合したパスを返す | パスオブジェクト |
| `with_name(name)` | `name` プロパティ値に該当する部分を、引数の `name` に変更したパスを返す。`name` プロパティ値が空文字列の<br /><br />場合は `ValueError` 例外が送出される | パスオブジェクト |
| `with_suffix(suffix)` | `suffix` プロパティ値に該当する部分を、引数の `suffix` に変更したパスを返す。`suffix` プロパティ値が空文<br /><br />字列の場合は、引数の `suffix` を追加する。引数の `suffix` が空文字列の場合、元の拡張子が削除される | パスオブジェクト |
| `with_stem(stem)` | `stem` プロパティ値に該当する部分を、引数の `stem` に変更したパスを返す。`steme` プロパティ値が空文字列の<br /><br />場合は `ValueError` 例外が送出される | パスオブジェクト |

これらのメソッドは、純粋パス `PurePath` から継承しており、システムコールを呼ばないので、`PurePosixPath` または `PureWindowsPath` のインスタンスメソッドでエミュレーションできる。

In [ ]:
from pathlib import Path

assert Path("/a/b").is_absolute()
assert Path("/spam/ham/eggs.py").is_relative_to("/spam")
assert not Path("/spam/ham/eggs.py").is_relative_to("/ham")
assert Path("/spam/ham/eggs.py").match("*.py")
assert Path("/spam").joinpath("ham", "eggs.py") == Path("/spam") / "ham" / "eggs.py" == Path("/spam/ham/eggs.py")
assert Path("/spam/ham/eggs.py").with_name("hoge.txt") == Path("/spam/ham/hoge.txt")
assert Path("/spam/ham/eggs.py").with_suffix(".txt") == Path("/spam/ham/eggs.txt")
assert Path("/spam/ham/eggs.py").with_suffix("") == Path("/spam/ham/eggs")
assert Path("/spam/ham/eggs.py").with_stem("foo") == Path("/spam/ham/foo.py")

`match()` メソッドの引数 `pattern` が相対表記であれば、パスは相対および絶対パスを取ることができ、右から一致を調べる:

In [ ]:
from pathlib import Path

assert Path("a/b.py").match("*.py")
assert Path("/a/b/c.py").match("b/*.py")
assert not Path("/a/b/c.py").match("a/*.py")

`match()` メソッドの引数 `pattern` が絶対表記であれば、パスは絶対パスでなければならず、パス全体が一致しなければならない:

In [ ]:
from pathlib import Path

assert Path('/a.py').match('/*.py')
assert not Path('a/b.py').match('/*.py')

### ファイルシステムの確認を行うメソッド

| メソッド | 機能 | 戻り値 |
|:---|:---|:---|
| `exists()` | パスが存在する場合 `True` を返す | `bool` |
| `is_dir(*, follow_symlinks=True)` | パスが無効・アクセス不能・存在しない場合、またはディレクトリ以外を指している場合は `False` を返<br /><br />す。このメソッドは通常はシンボリックリンクをたどる。Python 3.13 で `follow_symlinks` が追加さ<br /><br />れ、`False` を指定した場合、シンボリックリンクをたどらない | `bool` |
| `is_file(*, follow_symlinks=True)` | パスが無効・アクセス不能・存在しない場合、または通常のファイル以外を指している場合は `False`<br /><br /> を返す。このメソッドは通常はシンボリックリンクをたどる。Python 3.13 で `follow_symlinks` が追加<br /><br />され、`False` を指定した場合、シンボリックリンクをたどらない | `bool` |
| `is_symlink()` | パスがシンボリックリンクを指していた場合 `True` を返す | `bool` |
| `is_junction()` | パスがジャンクションを指している場合 `True` を返す。現在、Windows のみがジャンクションをサポー<br /><br />トしている。Python 3.12 で追加 | `bool` |
| `samefile(other_path)` | このパスが参照するファイルが `other_path`（`Path` オブジェクトか文字列）と同じであれば `True` を<br /><br />返す | `bool` |
| `stat(*, follow_symlinks=True)` | パスについての情報を ` os.stat_result` オブジェクトとして返す。`follow_symlinks` が偽ならシン<br /><br />ボリックリンクをたどらない | `os.stat_result` |

`is_*()` は、パスが存在していない場合、`False` を返す。

`stat()` メソッドが返す `os.stat_result` オブジェクトの主な属性は次のとおり。

| 属性 | 意味 |
|:---|:---|
| `st_mode` | ファイルモード。ファイルタイプとファイルモードのビット（権限） |
| `st_nlink` | ハードリンクの数 |
| `st_size` | ファイルのバイト単位でのサイズ |
| `st_atime` | 秒（POSIX タイムスタンプ）で表した最終アクセス時刻 |
| `st_mtime` | 秒（POSIX タイムスタンプ）で表した最終内容更新時刻 |
| `st_ctime` | 秒（POSIX タイムスタンプ）で表したメタデータの最終更新時刻（UNIX） / 作成時刻（Windows） |
| `st_birthtime` | 秒（POSIX タイムスタンプ）で表した作成時刻（UNIX）（Python 3.12 からは Windowsでも）  |

なお、Python 3.12 では、Windows 環境で `st_ctime` から作成時刻を取得することは非推奨になった。将来的には、UNIX 系と同様に、`st_ctime` にはメタデータの最終更新時刻が格納されるようになるとのこと。

In [ ]:
from pathlib import Path
from datetime import datetime

path = Path("sample_data/README.md")
assert path.exists()
assert path.parent.is_dir()
assert path.is_file()
print(f"{path.stat().st_size=}")
print(f"{path.name}の最終更新日時: {datetime.fromtimestamp(path.stat().st_mtime)}")

path.stat().st_size=930
README.mdの最終更新日時: 2000-01-01 08:00:00


### ジェネレーターの生成を行うメソッド

``` python
```

| メソッド | 機能 |
|:---|:---|
| `iterdir()` | パスがディレクトリを指していた場合、そのディレクトリ直下に存在する全てのファイルや<br /><br />ディレクトリをパスオブジェクトとして yield するジェネレーターを返す |
| `glob(pattern, *, case_sensitive=None, recurse_symlinks=False)` | パスがディレクトリを指していた場合、相対表記の `pattern` に一致する、そのディレクト<br /><br />リ以下の全てのファイルやディレクトリをパスオブジェクトとして yield するジェネレータ<br /><br />ーを返す。`case_sensitive` が `True` の場合は大文字小文字を区別し、`False` の場合<br /><br />は大文字小文字を区別せず、`None`（デフォルト）の場合は大文字小文字の区別がプラッ<br /><br />トフォームに依存する（Python 3.12 で追加）。`recurse_symlinks` が `True` の場合、シン<br /><br />ボリックリンクをたどる（Python 3.13 で追加）。 |
| `walk(top_down=True, on_error=None, follow_symlinks=False)` | Python 3.12 で追加。ディレクトリツリーを走査して、ディレクトリごとに、ディレクトリの<br /><br />パスオブジェクト、サブディレクトリ名のリスト、ファイル名のリストからなる 3 要素タプル<br /><br />を yield するジェネレーターを返す。`onerror` は走査中のエラー発生時に呼び出す関数<br /><br />を指定できる。この関数は引数として `OSError` インスタンスを伴って呼び出される。<br /><br />`follow_symlinks` に `True` を指定すると、シンボリックリンクの指しているディレクト<br /><br />リを走査する。シンボリックリンクが親ディレクトリを指していた場合に、無限ループにな<br /><br />ることに注意する |

`glob()` メソッドの引数 `pattern` には、glob パターンを指定できるほか、`**` も指定できる。`**` は、このディレクトリとすべてのサブディレクトリを再帰的に走査することを意味する。

In [ ]:
from pathlib import Path

print(f"{list(Path().iterdir())=}")
for p in Path().glob("**/*"):
    if p.suffix in (".yaml", ".md"):
        print(p, end=", ")

list(Path().iterdir())=[PosixPath('.config'), PosixPath('sample_data')]
.config/.last_opt_in_prompt.yaml, .config/.last_survey_prompt.yaml, sample_data/README.md, 

### パスオブジェクトの変換を行うメソッド

| メソッド | 機能 | 戻り値 |
|:---|:---|:---|
| `resolve(strict=False)` | パスを絶対パスにし、シンボリックリンクを解決する。パスが存在せず `strict` が `True` の場合、<br /><br />`FileNotFoundError` 例外を送出する。`strict` が`False` の場合は、パスは可能な限り解決され、<br /><br />残りの部分は存在するかのチェックをせずに追加される。もしパスの解決にあたって無限ループす<br /><br />る場合は、常に`RuntimeError` 例外を送出する（Python 3.13 では動作が変更され、無限ループす<br /><br />る場合、`strict` が `True` なら `OSError` を送出、`False` なら例外を送出しないようになった） | パスオブジェクト |
| `readlink()` | パスがシンボリックリンクの場合、シンボリックリンクが指すパスを返す | パスオブジェクト |

In [ ]:
from pathlib import Path

p = Path("sample_data/README.md")
print(p.resolve())

/content/sample_data/README.md


### ファイルやディレクトリの作成・変更・削除を行うメソッド

| メソッド | 機能 | 戻り値 |
|:---|:---|:---|
| `touch(mode=0o666, exist_ok=True)` | パスにファイルが存在しなければファイルを作成する。`mode` が与えられた場合、プロセスの<br /><br /> umask 値（Windows の場合 `0o000`?）と組み合わせてファイルのモードとアクセスフラグが<br /><br />決定される。ファイルが既に存在した場合、`exist_ok` が `True` なら更新日付を現在の日時<br /><br />に変更する。その他の場合は `FileExistsError` 例外が送出される | `None` |
| `mkdir(mode=0o777, parents=False, exist_ok=False)` | パスを新しいディレクトリとして作成する。`mode` が与えられていた場合、プロセスの umask <br /><br />値と組み合わせてモードとアクセスフラグが決定される。`parents` が `True` の場合、このパ<br /><br />スの親ディレクトリを必要に応じて作成する。`parents` が `False` の場合（デフォルト）、親<br /><br />ディレクトリがないと `FileNotFoundError` 例外を送出する。`exist_ok` が `False` の場合<br /><br />（デフォルト）、対象のディレクトリが既に存在すると `FileExistsError` を送出する。<br /><br />`exist_ok` が `True` の場合、パス要素の末尾が既に存在するがディレクトリではないときは<br /><br /> `FileExistsError` 例外を送出するが、ディレクトリであれば送出しない | `None` |
| `chmod(mode, *, follow_symlinks=True)` | Unix の chmod コマンドのようにパスのモードとアクセス権限を変更する | `None` |
| `rename(target)` | パスの名前を `target` に変更し、`target` を指すパスオブジェクトを返す。`target` が既存<br /><br />の場合、Windows 環境では常に `FileExistsError` 例外が送出されるが、Unix 環境ではユ<br /><br />ーザにパーミッションがあれば静かに置換される | パスオ<br />ブジェ<br />クト |
| `unlink(missing_ok=False)` | パスがファイル（かシンボリックリンク）を指していた場合、これを削除する。`missing_ok` が<br /><br /> `False` の場合（デフォルト）、対象のファイルが存在しないと `FileNotFoundError` 例外を送<br /><br />出する。`missing_ok` が `True` の場合、`FileExistsError` 例外を送出しない | `None` |
| `rmdir()` | パスがディレクトリを指していた場合、これを削除する。ディレクトリは空でなければならない  | `None` |
| `symlink_to(target, target_is_directory=False)` | このパスを `target` を指すシンボリックリンクにする。`target_is_directory` は Windows <br /><br />のみで有効な引数で、`True` ならディレクトリとして、`False` ならファイルとしてシンボリッ<br /><br />クリンクが作成される | `None` |

パスオブジェクトはパーミッション属性を持たないため、`chmod()` メソッドはオブジェクトの変更とはならない。

一方、`rename()` メソッドはパスに関する属性が異なる新しいパスオブジェクトを作成し、現在のパスオブジェクトを変更しない（パスオブジェクトはイミュータブルである）。

In [ ]:
!umask
from pathlib import Path
p1 = Path('text.bin')
p1.touch()  # 666 - 022 = 644 (所有者以外は書き込み不可)
!ls -l text.bin
p1.chmod(0o666)
p2 = p1.rename('foo.bin')
assert p2.name == 'foo.bin'
assert p1.name == 'text.bin'  # パスオブジェクトはイミュータブルであることに注意
!ls -l foo.bin
p2.unlink()

0022
-rw-r--r-- 1 root root 0 Aug 27 04:36 text.bin
-rw-rw-rw- 1 root root 0 Aug 27 04:36 foo.bin


### 例外

`pathlib.UnsupportedOperation` は、`NotImplementedError` を継承した例外である（Python 3.13 で追加）。パスオブジェクトがサポートしていない操作を行おうとした時に送出される。

低レイヤーなファイル操作
------------------------

標準ライブラリの `os` モジュールは、低レイヤーなファイル操作の機能を提供する。

ファイル操作関連の関数は、以下の通り。`path` 引数に渡すことができる値は、path-like オブジェクトである。

| 関数 | 機能 | 戻り値 | pathlib にある<br />同等の関数 |
|:---|:---|:--:|:---|
| `os.chmod(path, mode, *, dir_fd=None, follow_symlinks=True)` | Unix の chmod コマンドのように `path` のモードを変更する | `None` | `Path.chmod()` |
| `os.chown(path, uid, gid, *, dir_fd=None, follow_symlinks=True)` | Unix の chown コマンドのように `path` の所有者やグループ<br /><br />を変更する | `None` | なし |
| `os.chdir(path)` | 現在の作業ディレクトリを `path` に設定する | `None` | なし |
| `os.getcwd()` | 現在の作業ディレクトリを表す文字列を返す | `str` | `Path.cwd()` |
| `os.listdir(path='.')` | `path` に指定したディレクトリ内のファイルとサブディレクト<br /><br />リのリストを返す | `list` | `Path.iterdir()` |
| `os.walk(top, topdown=True, onerror=None, followlinks=False)` | ディレクトリ `top` を根に持つディレクトリツリーに含まれる、<br /><br />各ディレクトリ（`top` 自身を含む）ごとに、3 要素タプル<br /><br /> `(dirpath, dirnames, filenames)` を yield する。<br /><br />・`dirpath`: ディレクトリのパス<br /><br />・`dirnames`: `dirpath` 内のサブディレクトリの名前のリスト<br /><br />　（ディレクトリへのシンボリックリンクを含み、`.` と `..` を<br /><br />　除く）<br /><br />・`filenames`: `dirpath` 内のディレクトリ以外のファイルの名<br /><br />　前のリスト<br /><br />オプション引数は次のとおり<br /><br />・`topdown` が `True`（デフォルト）の場合、サブディレクトリか<br /><br />　らタプルを生成する<br /><br />・`onerror` には、走査中のエラー発生時に呼び出す関数を指<br /><br />　定できる。この関数は引数として`OSError` インスタンスを<br /><br />　伴って呼び出される<br /><br />・`followlinks` に `True` を指定すると、シンボリックリンク<br /><br />　の指しているディレクトリを走査する。シンボリックリンクが<br /><br />　親ディレクトリを指していた場合は無限ループになる |  ｼﾞｪﾈ<br />ﾚｰﾀｰ | `Path.walk()` |
| `os.mkdir(path, mode=0o777, *, dir_fd=None)` | `path` に指定したディレクトリを作成する | `None` | `Path.mkdir()` |
| `os.makedirs(name, mode=0o777, exist_ok=False)` | `name='dir1/dir2/dir3'` のような深い階層のパスを、中間<br /><br />ディレクトリも含めて一気に新規作成する | `None` | `Path.mkdir()` |
| `os.rmdir(path, *, dir_fd=None)` | `path` に指定したディレクトリを削除する。ディレクトリが空で<br /><br />ない場合は `OSError` 例外を送出する | `None` | `Path.rmdir()` |
| `os.removedirs(name)` | `name='dir1/dir2/dir3'` のような深い階層のパスに対して、<br /><br />空のディレクトリを一番下の階層から順番に削除する。空でな<br /><br />いディレクトリは削除されない | `None` | なし |
| `os.remove(path, *, dir_fd=None)` | `path` に指定したファイルを削除する。ディレクトリの場合は<br /><br /> `OSError` 例外を送出する | `None` | `Path.unlink()` |
| `os.unlink(path, *, dir_fd=None)` | `os.remove()` と同等 | `None` | `Path.unlink()` |
| `os.rename(src, dst, *, src_dir_fd=None, dst_dir_fd=None)` | ファイルまたはディレクトリのパスを `src` から `dst` に変更<br /><br />する。`dst` が既存の場合、以下のようになる<br /><br />・Windows 環境では、常に `FileExistsError` が発生する<br /><br />・Unix 環境では、 `src` と `dst` がともにディレクトリで `dst` <br /><br />　が空なら静かに置換される。また、両方がファイルで、ユー<br /><br />　ザーにパーミッションがあれば静かに置換される。それ以外<br /><br />　では例外が発生する | `None` | `Path.rename()` |
| `os.renames(old, new)` | `old` に指定したファイルまたはディレクトリを、`new` に指定<br /><br />した深い階層のパスに移動する。中間ディレクトリが存在しな<br /><br />い場合に自動的に作成する | `None` | なし |
| `os.symlink(src, dst, target_is_directory=False, *, dir_fd=None)` | `src` を指し示すシンボリックリンク `dst` を作成する | `None` | `Path.symlink_to()` |

また、`os` モジュールはファイルに関する定数を提供している。

| 定数 | 意味 |
|:---|:---|
| `os.curdir` | 現在のディレクトリ参照するために OS で使われる文字列定数。POSIX と Windows では `'.'` |
| `os.pardir` | 親ディレクトリを参照するために OS で使われる文字列定数。POSIX と Windows では `'..'` |
| `os.sep` | パス名を要素に分割するために OS で利用されている文字。POSIX では `'/'` で、 Windows では `'\\'` |
| `os.extsep` | ベースのファイル名と拡張子を分ける文字。たとえば、`os.py` であれば `'.'` のこと |
| `os.linesep` | 行の終端を表すために OS で利用されている文字列。POSIX では `'\n'`、Mac OS では `'\r'`、Windows では `'\r\n'` |
| `os.devnull` | ヌルデバイスのファイルパス。POSIX では `'/dev/null'` で、 Windows では `'nul'` |

標準ライブラリの `os.path` モジュールは、パス名に関するいくつかの便利な関数を提供している。`path` 引数に渡すことができる値は、path-like オブジェクトである。

| 関数 | 機能 | 戻り値 | pathlib にある<br />同等の関数等 |
|:---|:---|:---|:---|
| `os.path.split(path)` | パス名 `path` を `(head, tail)` のペアに分割する。`tail` はパス名の構成要素の末尾<br /><br />で、`head` はそれより前の部分となる。もし `path` がスラッシュで終わっていれば `tail`<br /><br /> は空文字列になる。もし `path` にスラッシュがなければ、`head` は空文字になる。`path`<br /><br /> が空文字なら、`head` と `tail` の両方が空文字になる。`head `の末尾のスラッシュは<br /><br /> `head` がルートディレクトリ（または 1 個以上のスラッシュだけ）でない限り取り除かれる | `tuple` | なし |
| `os.path.basename(path)` | `os.path.split(path)[1]` と同じ | `str` | `Path.name` |
| `os.path.dirname(path)` | `os.path.split(path)[0]` と同じ | `str` | `Path.parent` |
| `os.path.join(path, *paths)` | 引数を構成要素とするパス名を返す。`os.path.split()` の逆操作となる | `str` | `Path.joinpath() ` |
| `os.path.normpath(path)` | パスを正規化する。すなわち、余分な区切り文字や上位レベル参照を除去するので、<br /><br />`A//B`、`A/B/`、`A/./B` や `A/foo/../B` などはすべて `A/B` になる。この文字列操作は、<br /><br />シンボリックリンクを含むパスの意味を変えてしまう場合がある。Windows ではスラッ<br /><br />シュをバックスラッシュに変換する | `str` | なし |
| `os.path.abspath(path)` | パス名 `path` の正規化された絶対パスを返す。<br /><br />`normpath(join(os.getcwd(), path))` と等価である | `str` | なし |
| `os.path.relpath(path,`<br />` start=os.curdir)` | `start` ディレクトリを基準とする `path` の相対パスを返す。`start` のデフォルトはカレ<br /><br />ントディレクトリ | `str` | なし |
| `os.path.expanduser(path)` | Unix および Windows では、与えられた引数の先頭のパス要素 `~` 、または `~user` を、<br /><br />`user` のホームディレクトリのパスに置き換えて返す。Windows では、環境変数<br /><br /> `USERPROFILE` が使用される。`USERPROFILE` が設定されていない場合は環境変数<br /><br /> `HOMEPATH` と `HOMEDRIVE` の組み合わせが使用される | `str` | なし |
| `os.path.expandvars(path)` | 引数を、環境変数を展開して返す。引数の中の `$name` または `${name}` のような形式<br /><br />の文字列は環境変数、`name` の値に置き換えられる。不正な変数名や存在しない変数名<br /><br />の場合には変換されず、そのまま返す | `str` | なし |
| `os.path.commonpath(paths)` | パス名のシーケンス `paths` 中の共通するサブパスのうち、最も長いものを返す。`paths`<br /><br /> に絶対パス名と相対パス名の両方が含まれている場合や、`paths` が異なるドライブ上に<br /><br />ある場合や、`paths` が空の場合、`ValueError` 例外を送出する | `str` | なし |
| `os.path.exists(path)` | `path` が存在する場合 `True` を返す | `bool` | `Path.exists()` |
| `os.path.isabs(path)` | `path` が絶対パスなら `True` を返す | `bool` | `Path.is_absolute()` |
| `os.path.isfile(path)` | `path` が存在するファイルなら `True` を返す | `bool` | `Path.is_file()` |
| `os.path.isdir(path)` | `path` が存在するディレクトリなら `True` を返す | `bool` | `Path.is_dir()` |
| `os.path.islink(path)` | `path` が存在するディレクトリを指すシンボリックリンクなら `True` を返す。シンボリック<br /><br />リンクをサポートしていないプラットフォームでは、常に `False` を返す | `bool` | `Path.is_symlink()` |
| `os.path.samefile(path1, path2)` | 引数の両パス名が同じファイルまたはディレクトリを参照している場合に `True` を返す | `bool` | `Path.samefile()` |
| `os.path.getatime(path)` | `path` に最後にアクセスした時刻（エポック秒）を返す | `float` | `Path.stat()` |
| `os.path.getmtime(path)` | `path` に最後に更新した時刻（エポック秒）を返す | `float` | `Path.stat()` |
| `os.path.getctime(path)` | Unix 系では最後に`path` のメタデータが変更された時刻、Windows では `path` の作成<br /><br />時刻を返す。返り値はエポック秒 | `float` | `Path.stat()` |
| `os.path.getsize(path)` | `path` のサイズをバイト数で返す | `int` | `Path.stat()` |

`get*()` 関数は、ファイルが存在しないか、あるいはアクセスできなかった場合は `OSError` 例外を送出する。

Python 3.11 以前では `Path.walk()` メソッドがサポートされないので、ディレクトリを再帰的に走査するのに `os.walk()` 関数を使うことになる。`dirpath` 内のファイルまたはディレクトリへの完全パス（`top` で始まる）を取得するには、`os.path.join(dirpath, name)` を実行する。

In [ ]:
import os

def list_files_recursively(directory):
    for cudir, dirs, files in os.walk(directory):
        for file in files:
            print(os.path.join(cudir, file))

list_files_recursively('.')

./.config/.last_opt_in_prompt.yaml
./.config/default_configs.db
./.config/config_sentinel
./.config/gce
./.config/.last_update_check.json
./.config/.last_survey_prompt.yaml
./.config/hidden_gcloud_config_universe_descriptor_data_cache_configs.db
./.config/active_config
./.config/configurations/config_default
./.config/logs/2024.08.23/13.19.51.397286.log
./.config/logs/2024.08.23/13.19.16.438002.log
./.config/logs/2024.08.23/13.19.38.729140.log
./.config/logs/2024.08.23/13.20.05.747593.log
./.config/logs/2024.08.23/13.19.52.608801.log
./.config/logs/2024.08.23/13.20.05.018217.log
./sample_data/anscombe.json
./sample_data/README.md
./sample_data/california_housing_train.csv
./sample_data/mnist_test.csv
./sample_data/mnist_train_small.csv
./sample_data/california_housing_test.csv
